In [10]:
from scipy.optimize import minimize

loading data from the sql 

In [11]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

# --- PostgreSQL Connection ---
engine = create_engine('postgresql://akilfiros:@127.0.0.1:5432/postgres')

# --- Load historical price data ---
query = "SELECT date, ticker, close FROM financial_data"
df = pd.read_sql(query, engine)
df['date'] = pd.to_datetime(df['date'])

# --- Pivot to get price matrix [date x ticker] ---
price_df = df.pivot(index='date', columns='ticker', values='close').sort_index()
price_df = price_df.dropna(axis=1)  # Drop tickers with missing data

# --- Compute log returns ---
log_returns = np.log(price_df / price_df.shift(1)).dropna()

# ✅ Preview
print("Log returns shape:", log_returns.shape)
print(log_returns.head())


Log returns shape: (1333, 92)
ticker          AAPL      ADBE       ADI       ADP       AEP      ALGN  \
date                                                                     
2020-01-03 -0.009770 -0.007865 -0.017760 -0.002116 -0.001071 -0.011487   
2020-01-06  0.007937  0.005710 -0.011818  0.001352  0.003315  0.019212   
2020-01-07 -0.004714 -0.000959  0.022496 -0.012178  0.000213 -0.009913   
2020-01-08  0.015958  0.013348  0.008991  0.009354 -0.002993  0.010333   
2020-01-09  0.021019  0.007607  0.000000  0.008800  0.005019  0.036190   

ticker          AMAT       AMD      AMGN      AMZN  ...      TEAM      TSLA  \
date                                                ...                       
2020-01-03 -0.016044 -0.010236 -0.006812 -0.012213  ...  0.001301  0.029203   
2020-01-06 -0.021801 -0.004330  0.007645  0.014776  ...  0.037096  0.019072   
2020-01-07  0.028477 -0.002897 -0.009449  0.002089  ...  0.003596  0.038067   
2020-01-08 -0.000649 -0.008743  0.000756 -0.007839  ... 

calculating expected returns & covariance matirx 

In [12]:
# Assuming 252 trading days per year
expected_returns = log_returns.mean() * 252

# --- Compute annualized covariance matrix ---
cov_matrix = log_returns.cov() * 252

# --- Preview ---
print("✅ Expected Returns (annualized):")
print(expected_returns.head())

print("\n✅ Covariance Matrix (annualized):")
print(cov_matrix.iloc[:5, :5])  # Show a 5x5 slice


✅ Expected Returns (annualized):
ticker
AAPL    0.195568
ADBE    0.009658
ADI     0.097311
ADP     0.123929
AEP     0.061595
dtype: float64

✅ Covariance Matrix (annualized):
ticker      AAPL      ADBE       ADI       ADP       AEP
ticker                                                  
AAPL    0.107306  0.078754  0.074804  0.049564  0.026940
ADBE    0.078754  0.153997  0.081360  0.053288  0.020636
ADI     0.074804  0.081360  0.136307  0.056034  0.022406
ADP     0.049564  0.053288  0.056034  0.076400  0.034079
AEP     0.026940  0.020636  0.022406  0.034079  0.061350


In [13]:
# --- Prepare data ---
tickers = expected_returns.index.tolist()
mu = expected_returns.values
cov = cov_matrix.values
n_assets = len(tickers)

# --- Objective function: portfolio variance ---
def portfolio_variance(weights, cov_matrix):
    return weights.T @ cov_matrix @ weights

# --- Constraints: fully invested ---
constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]

# --- Bounds: long-only portfolio (0 <= w <= 1) ---
bounds = [(0.0, 1.0) for _ in range(n_assets)]

# --- Initial guess: equal weighting ---
w0 = np.ones(n_assets) / n_assets

# --- Solve optimization ---
result = minimize(portfolio_variance, w0, args=(cov,), method='SLSQP',
                  bounds=bounds, constraints=constraints)

# --- Extract optimal weights ---
optimal_weights = pd.Series(result.x, index=tickers)

# --- Display result ---
print("✅ Optimal Weights (Minimum Variance Portfolio):")
print(optimal_weights[optimal_weights > 0].sort_values(ascending=False))


✅ Optimal Weights (Minimum Variance Portfolio):
KDP      1.797229e-01
GILD     1.425477e-01
AEP      1.262701e-01
AZN      1.039419e-01
COST     1.005409e-01
EA       9.508363e-02
PCAR     5.621279e-02
REGN     3.336275e-02
MAR      2.981267e-02
NTES     2.363510e-02
MDLZ     2.230512e-02
KHC      1.949568e-02
TTWO     1.881642e-02
TCOM     1.729492e-02
ORLY     1.488574e-02
DLTR     9.929543e-03
PANW     4.405840e-03
AMGN     1.736242e-03
ADI      6.529003e-17
ALGN     3.949031e-17
ADBE     2.949523e-17
GOOG     2.675874e-17
TSLA     2.277339e-17
KLAC     1.699253e-17
CTAS     1.103287e-17
INTC     1.035093e-17
NXPI     1.034166e-17
SNPS     1.028934e-17
SMCI     9.472281e-18
AXON     9.140418e-18
QCOM     8.211398e-18
MU       8.100045e-18
PEP      7.024982e-18
DDOG     6.753539e-18
VRSN     6.367585e-18
DXCM     5.865500e-18
MSFT     5.643939e-18
ODFL     5.526984e-18
XEL      4.811667e-18
NVDA     4.542031e-18
MTCH     3.198064e-18
WBA      3.086078e-18
FTNT     2.876317e-18
EXC   


- These are the optimal asset weights (allocations) that minimize the total portfolio variance, assuming:
    - Full investment (sum = 1)
    - No short selling (weights ≥ 0)
- Assets with very small values (e.g., 1e-17) are essentially excluded from the portfolio — their weight is effectively zero.

Calculating for Constrained Portfolio Optimization (Target Return + Max Weight)

In [14]:
# --- Target return: e.g., at least 5% annual return ---
target_return = 0.05

# --- Objective: minimize portfolio variance ---
def portfolio_variance(weights, cov_matrix):
    return weights.T @ cov_matrix @ weights

# --- Constraints: fully invested + target return ---
constraints = [
    {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # weights sum to 1
    {'type': 'ineq', 'fun': lambda w: w @ mu - target_return}  # return constraint
]

# --- Bounds: long-only + max 10% in any asset ---
max_weight = 0.10
bounds = [(0.0, max_weight) for _ in range(n_assets)]

# --- Initial guess ---
w0 = np.ones(n_assets) / n_assets

# --- Solve ---
result_constrained = minimize(portfolio_variance, w0, args=(cov,), method='SLSQP',
                              bounds=bounds, constraints=constraints)

# --- Extract optimal weights ---
optimal_weights_constrained = pd.Series(result_constrained.x, index=tickers)

# --- Display non-zero weights ---
print("✅ Constrained Optimal Weights:")
print(optimal_weights_constrained[optimal_weights_constrained > 0].sort_values(ascending=False))

✅ Constrained Optimal Weights:
COST     1.000000e-01
AEP      1.000000e-01
KDP      1.000000e-01
GILD     1.000000e-01
AZN      1.000000e-01
EA       1.000000e-01
PCAR     6.697239e-02
MDLZ     6.463593e-02
REGN     5.048294e-02
KHC      4.412045e-02
MAR      3.365828e-02
ORLY     2.962573e-02
NTES     2.698515e-02
TTWO     2.237375e-02
AMGN     2.156399e-02
TCOM     1.817852e-02
DLTR     1.426197e-02
PANW     5.504696e-03
XEL      1.315616e-03
WBA      3.205777e-04
AMAT     5.277493e-17
MELI     3.574802e-17
PEP      3.490911e-17
MCHP     3.291873e-17
ADI      3.013572e-17
CTSH     2.634285e-17
ILMN     2.259208e-17
INTU     2.218804e-17
KLAC     2.198793e-17
BIDU     1.912670e-17
AAPL     1.634000e-17
QCOM     1.495951e-17
MSFT     1.246230e-17
PDD      9.656648e-18
MTCH     9.286499e-18
CRWD     8.318921e-18
NVDA     6.519116e-18
IDXX     5.396356e-18
NXPI     3.987341e-18
ISRG     3.240346e-18
DOCU     3.022688e-18
DXCM     2.395128e-18
DDOG     2.011807e-18
CDNS     1.937632e-18
O


- Enforced param:
    - Target return ≥ 5%
    - Max weight = 10% per asset
    - Long-only constraint (no negative weights)
- As a result:
    - Assets like EA, AZN, KDP, AEP, and GILD hit the maximum allowed allocation of 10% — they are likely low-risk, high-return contributors.
    - Several other assets have near-zero weights (e.g., MRVL, MSTR) — the optimizer deems them suboptimal under your constraints.


Saving connstrained weights to the sql 


In [15]:
from sqlalchemy import create_engine

# --- Prepare final weights table ---
weights_df = optimal_weights_constrained.reset_index()
weights_df.columns = ['ticker', 'weight']
weights_df = weights_df[weights_df['weight'] > 0].sort_values(by='weight', ascending=False)

# --- Save to PostgreSQL ---
engine = create_engine('postgresql://akilfiros:@127.0.0.1:5432/postgres')
weights_df.to_sql("optimal_portfolio_weights", engine, if_exists='replace', index=False)

print("✅ Optimal weights saved to PostgreSQL table: optimal_portfolio_weights")

✅ Optimal weights saved to PostgreSQL table: optimal_portfolio_weights
